# Preprocessing .mat for NWB

In [1]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import LO_NWB_conv



# Load data 

## mat

In [28]:
import scipy.io
import h5py

filename = "data/AO039_20190626.mat"  

with h5py.File(filename, 'r') as f:
    data_group = f['Data']
    data_ref = f["#refs#"]
    #print("Références trouvées :")
    #for key in data_ref.keys():
    #    print(f" - {key} → shape: {data_ref[key].shape}, dtype: {data_ref[key].dtype}")
    print("Contenu de 'Data' :")
    for key in data_group.keys():
        print(f" - {key} → shape: {data_group[key].shape}, dtype: {data_group[key].dtype}")


Contenu de 'Data' :
 - ARAindex → shape: (75, 1), dtype: float64
 - Area → shape: (75, 1), dtype: object
 - BaselineFR_Mean → shape: (75, 1), dtype: float64
 - BaselineFR_Session → shape: (75, 1), dtype: object
 - CRIndices → shape: (1, 583), dtype: uint8
 - ClusterCounter → shape: (75, 1), dtype: float64
 - EngagedTrials → shape: (583, 1), dtype: uint8
 - FAIndices → shape: (1, 583), dtype: uint8
 - HitIndices → shape: (1, 583), dtype: uint8
 - ISI_Violation → shape: (75, 1), dtype: float64
 - ISO_Distance → shape: (75, 1), dtype: float64
 - JawOnsetsTms → shape: (583, 1), dtype: float64
 - JawTrace → shape: (583, 1000), dtype: float64
 - LFPs → shape: (3, 1), dtype: object
 - LickData → shape: (482142, 1), dtype: float64
 - LickTime → shape: (482142, 1), dtype: float64
 - LightIndices → shape: (1, 583), dtype: float64
 - MDS → shape: (1, 1), dtype: float64
 - ML_DV_AP → shape: (75, 1), dtype: object
 - ML_DV_AP_32 → shape: (75, 1), dtype: object
 - MissIndices → shape: (1, 583), dtyp

In [29]:
with h5py.File(filename, 'r') as f:
	data_group = f['Data'] if 'Data' in f else f
	data = {key: data_group[key][()] for key in data_group.keys()}


In [80]:

hit = np.asarray(data['HitIndices']).flatten().astype(bool)
miss = np.asarray(data['MissIndices']).flatten().astype(bool)
cr = np.asarray(data['CRIndices']).flatten().astype(bool)
fa = np.asarray(data['FAIndices']).flatten().astype(bool)

n_trials = len(hit)
response_labels = np.full(n_trials, 'Unlabeled', dtype=object)  # valeur par défaut

# Attribution avec priorité : FA < CR < MISS < HIT
response_labels[fa] = 'FA'
response_labels[cr] = 'CR'
response_labels[miss] = 'MISS'
response_labels[hit] = 'HIT'

labels = ['HIT', 'MISS', 'CR', 'FA', 'Unlabeled']
label_to_int = {label: i for i, label in enumerate(labels)}

response_data = np.array([label_to_int[label] for label in response_labels])
response_data.

array([0, 0, 2, 2, 0, 2, 0, 2, 1, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 1, 0,
       2, 0, 2, 0, 0, 2, 2, 1, 3, 0, 2, 0, 3, 0, 2, 2, 0, 2, 0, 2, 0, 0,
       2, 3, 1, 2, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 1,
       0, 0, 2, 3, 1, 2, 0, 3, 2, 1, 1, 0, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2,
       0, 0, 3, 2, 3, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0,
       2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 1, 2, 0, 2, 2, 0, 0, 0, 0, 0,
       2, 2, 0, 1, 2, 0, 0, 2, 2, 2, 0, 1, 2, 2, 2, 0, 3, 3, 0, 0, 0, 3,
       2, 2, 1, 1, 2, 0, 0, 2, 0, 2, 2, 3, 0, 0, 3, 2, 1, 2, 0, 2, 2, 0,
       0, 2, 1, 1, 0, 2, 2, 2, 0, 4, 0, 2, 0, 2, 2, 2, 1, 1, 1, 2, 2, 2,
       2, 2, 0, 1, 2, 2, 1, 1, 2, 1, 0, 2, 0, 2, 2, 2, 2, 0, 1, 0, 0, 2,
       2, 1, 2, 0, 2, 2, 1, 1, 2, 1, 2, 3, 2, 1, 1, 2, 2, 0, 2, 4, 0, 0,
       0, 2, 2, 2, 1, 0, 0, 3, 2, 3, 2, 0, 0, 2, 2, 0, 2, 2, 1, 0, 2, 0,
       2, 2, 0, 0, 1, 0, 2, 1, 2, 2, 2, 1, 2, 4, 2, 0, 0, 2, 0, 2, 1, 2,
       1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 0, 0, 1, 1,

In [82]:
import collections
print(collections.Counter(response_labels))
print(hit.sum(), miss.sum(), cr.sum(), fa.sum(), response_labels.size)

Counter({'CR': 271, 'HIT': 150, 'MISS': 136, 'FA': 17, 'Unlabeled': 9})
150 136 271 17 583


In [76]:
print(hit.sum())
print(miss.sum())
print(cr.sum())
print(fa.sum())
print(response_labels.size)

[ True  True False False  True False  True False False False  True False
 False  True  True False False  True False False False  True False  True
 False  True  True False False False False  True False  True False  True
 False False  True False  True False  True  True False False False False
  True  True  True False False  True False  True  True False False False
 False  True False False False False  True  True False False False False
  True False False False False  True False False False  True  True  True
 False  True False False  True  True False False False  True  True  True
 False False  True False  True False False False  True  True  True False
 False  True False  True False False  True  True False False False  True
  True False False False  True False False  True  True  True  True  True
 False False  True False False  True  True False False False  True False
 False False False  True False False  True  True  True False False False
 False False False  True  True False  True False Fa

array(['HIT', 'HIT', 'CR', 'CR', 'HIT', 'CR', 'HIT', 'CR', 'MISS', 'CR',
       'HIT', 'CR', 'CR', 'HIT', 'HIT', 'CR', 'CR', 'HIT', 'CR', 'CR',
       'MISS', 'HIT', 'CR', 'HIT', 'CR', 'HIT', 'HIT', 'CR', 'CR', 'MISS',
       'FA', 'HIT', 'CR', 'HIT', 'FA', 'HIT', 'CR', 'CR', 'HIT', 'CR',
       'HIT', 'CR', 'HIT', 'HIT', 'CR', 'FA', 'MISS', 'CR', 'HIT', 'HIT',
       'HIT', 'CR', 'CR', 'HIT', 'CR', 'HIT', 'HIT', 'CR', 'CR', 'CR',
       'CR', 'HIT', 'CR', 'CR', 'CR', 'MISS', 'HIT', 'HIT', 'CR', 'FA',
       'MISS', 'CR', 'HIT', 'FA', 'CR', 'MISS', 'MISS', 'HIT', 'CR', 'CR',
       'CR', 'HIT', 'HIT', 'HIT', 'CR', 'HIT', 'CR', 'CR', 'HIT', 'HIT',
       'FA', 'CR', 'FA', 'HIT', 'HIT', 'HIT', 'CR', 'CR', 'HIT', 'CR',
       'HIT', 'CR', 'CR', 'CR', 'HIT', 'HIT', 'HIT', 'CR', 'CR', 'HIT',
       'CR', 'HIT', 'CR', 'CR', 'HIT', 'HIT', 'CR', 'CR', 'CR', 'HIT',
       'HIT', 'CR', 'MISS', 'CR', 'HIT', 'CR', 'CR', 'HIT', 'HIT', 'HIT',
       'HIT', 'HIT', 'CR', 'CR', 'HIT', 'MISS', 'CR', 'HI

In [63]:
trace = np.array(data["JawTrace"])
print(trace)
trace.shape
full_trace = trace.flatten()
full_trace

[[ 165.81881573  163.1152512   163.29824412 ... 1095.61013355
  1062.94662208 1036.20043938]
 [  63.54225859   69.11485662   70.06473559 ... 1280.42382492
  1322.21517446 1367.21601979]
 [  48.76324753   47.71333557   47.57579206 ...  221.26881635
   217.35100238  222.09739779]
 ...
 [ 296.02685719  295.7401318   289.54849353 ...   94.44972364
    99.18861739  109.31491614]
 [ 224.14464319  213.74054738  210.70358747 ...  160.97787938
   154.4583203   156.90013156]
 [ 525.43188921  517.61611846  516.09763218 ...  503.98008819
   505.85341787  509.66784086]]


array([165.81881573, 163.1152512 , 163.29824412, ..., 503.98008819,
       505.85341787, 509.66784086])

## CSV


In [178]:

csv_data = pd.read_csv("data/Subject_Session_Selection.csv", sep=";")
csv_data.columns = csv_data.columns.str.strip() 
csv_data.columns

mouse = ''.join(chr(c) for c in data['mouse'].flatten())
date = ''.join(chr(c) for c in data['date'].flatten())
session_name = f"{mouse}_{date}"  

try:
    subject_info = csv_data[csv_data['Session'].astype(str).str.strip() == session_name].iloc[0]
except IndexError:
    raise ValueError(f"Session {session_name} not found in the CSV file.")

 # TOTAL *******

In [86]:
# Supprimer tous les fichiers dans le dossier NWB output
for file in os.listdir("data/nwb_output"):
    file_path = os.path.join("data/nwb_output", file)
    if os.path.isfile(file_path):
        os.remove(file_path)
print("Tous les fichiers ont été supprimés du dossier data/nwb_output.")

filename = "data/AO039_20190626.mat"  # Chemin vers le fichier .mat
importlib.reload(LO_NWB_conv)
nwb_path = LO_NWB_conv.convert_data_to_nwb_an(mat_file=filename, output_folder="data/nwb_output", output_folder_config="data")

Tous les fichiers ont été supprimés du dossier data/nwb_output.
**************************************************************************
-_-_-_-_-_-_-_-_-_-_-_-_-_-_- NWB conversion _-_-_-_-_-_-_-_-_-_-_-_-_-_-_
 
📃 Creating config file for NWB conversion :
   - data/AO039_20190626_config.yaml
 
📑 Created NWB file
     o 🐭 Subject metadata & Session metadata
     o 💬 Add behavior container
 
🔎 Validating NWB file before saving...
     o ✅ File is valid, no errors detected.
 
💾 Saving NWB file
     o 📂 NWB file saved at:
         - data/nwb_output/AO039_20190626_160524_2025_07_10.17-54-00.nwb
**************************************************************************


# Visualization

In [4]:
from pynwb import NWBHDF5IO

io = NWBHDF5IO(nwb_path, 'r')
nwbfile = io.read()

In [22]:
ts = nwbfile.processing['behavior']['BehavioralEvents']['TrialOnsets']

In [26]:
ts.timestamps[:]

array([  15.30276667,   22.1426    ,   27.7827    ,   34.59283333,
         46.34316667,   53.99356667,   64.0133    ,   72.27296667,
         79.60266667,   90.32283333,   98.75276667,  104.11266667,
        113.873     ,  125.05736667,  131.30276667,  138.0229    ,
        148.56293333,  158.72296667,  164.95733333,  180.13293333,
        189.91306667,  198.27303333,  208.9927    ,  217.94303333,
        224.78276667,  233.273     ,  238.90293333,  245.35276667,
        252.53316667,  263.1129    ,  272.73283333,  279.78283333,
        286.85283333,  294.753     ,  302.53293333,  309.1329    ,
        317.9431    ,  326.3229    ,  337.4327    ,  345.08273333,
        352.87253333,  359.49276667,  367.0427    ,  373.4726    ,
        379.14286667,  389.14283333,  397.17263333,  407.2327    ,
        418.95266667,  425.83286667,  434.06266667,  445.2028    ,
        457.6225    ,  464.7726    ,  471.41253333,  482.59246667,
        492.4526    ,  502.02266667,  515.68266667,  522.5726 